### <h1>Segmenting and Clustering Neighborhoods in Toronto - 3</h1>

Import necessary libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')


Libraries imported.


Download and Explore Dataset
Use the csv file to create the dataframe: here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [2]:
df_gc = pd.read_csv('http://cocl.us/Geospatial_data')

Obtain the data that is in the table of postal codes in the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M  and to transform the data into a pandas dataframe.

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_list = pd.read_html(url)
df_bn=df_list[0]

Split text in Neighborhood column into multiple rows

In [4]:
#first define a function: given a Series of string, split each element into a new series
def split_series(ser,sep):
    return pd.Series(ser.str.cat(sep=sep).split(sep=sep)) 


In [5]:
#Split Neighbourhood column
df_bn2=(df_bn.groupby(df_bn.columns.drop('Neighbourhood').tolist()) #group by all but one column
          ['Neighbourhood'] #select the column to be split
          .apply(split_series,sep=', ') # split 'Neighbourhood' in each group
         .reset_index(drop=True,level=-1).reset_index()) #remove extra index created
df_bn2.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,Malvern
2,M1B,Scarborough,Rouge
3,M1C,Scarborough,Rouge Hill
4,M1C,Scarborough,Port Union


Merge 2 dataframes with 'Postal Code' as a key.

In [57]:
neighborhoods_all = pd.merge(df_bn2, df_gc, on='Postal Code')

Include Borough that contains Toronto

In [62]:
neighborhoods = neighborhoods_all[neighborhoods_all ['Borough'].str.contains('Toronto')]
neighborhoods

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
68,M4E,East Toronto,The Beaches,43.676357,-79.293031
73,M4K,East Toronto,The Danforth West,43.679557,-79.352188
74,M4K,East Toronto,Riverdale,43.679557,-79.352188
75,M4L,East Toronto,India Bazaar,43.668999,-79.315572
76,M4L,East Toronto,The Beaches West,43.668999,-79.315572
77,M4M,East Toronto,Studio District,43.659526,-79.340923
78,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
79,M4P,Central Toronto,Davisville North,43.712751,-79.390197
80,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
81,M4R,Central Toronto,Lawrence Park,43.715383,-79.405678


Define Foursquare Credentials and Version

In [59]:
CLIENT_ID = 'EBR4IH4AB13WLISP1YTJFQFVHWJIF3MCYDHQ2IFQHK1UEIF1' # your Foursquare ID
CLIENT_SECRET = 'TMSYIKCPEAHWZMB4NEXUGSWEHLIIKD3M3JNFEUPJMSDUZH0X' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EBR4IH4AB13WLISP1YTJFQFVHWJIF3MCYDHQ2IFQHK1UEIF1
CLIENT_SECRET:TMSYIKCPEAHWZMB4NEXUGSWEHLIIKD3M3JNFEUPJMSDUZH0X


Define get category value function

In [11]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a _pandas_ dataframe.

In [64]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056


 create a function to repeat the same process to all the neighborhoods

In [65]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called _toronto_venues_.

In [82]:
toronto_venues = getNearbyVenues(neighborhoods['Neighbourhood'], neighborhoods['Latitude'],neighborhoods['Longitude'])

The Beaches
The Danforth West
Riverdale
India Bazaar
The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Lawrence Park
Davisville
Moore Park
Summerhill East
Summerhill West
Rathnelly
South Hill
Forest Hill SE
Deer Park
Rosedale
St. James Town
Cabbagetown
Church and Wellesley
Regent Park
Harbourfront
Garden District
Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond
Adelaide
King
Harbourfront East
Union Station
Toronto Islands
Toronto Dominion Centre
Design Exchange
Commerce Court
Victoria Hotel
Roselawn
Forest Hill North & West
Forest Hill Road Park
The Annex
North Midtown
Yorkville
University of Toronto
Harbord
Kensington Market
Chinatown
Grange Park
CN Tower
King and Spadina
Railway Lands
Harbourfront West
Bathurst Quay
South Niagara
Island airport
Stn A PO Boxes
First Canadian Place
Underground city
Christie
Dufferin
Dovercourt Village
Little Portugal
Trinity
Brockton
Parkdale Village
Exhibition Place
High Park
The Junction South
Parkdale
R

In [85]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Danforth West,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


## Analyze Each Neighborhood

In [86]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] =toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [87]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Adelaide,0.0,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.02,...,0.01,0.0,0.0,0.0,0.0,0.010000,0.0,0.0,0.0,0.01
1,Bathurst Quay,0.0,0.0,0.0625,0.0625,0.0625,0.125,0.125,0.0625,0.00,...,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00
2,Berczy Park,0.0,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.00,...,0.00,0.0,0.0,0.0,0.0,0.018182,0.0,0.0,0.0,0.00
3,Brockton,0.0,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.00,...,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00
4,Business reply mail Processing Centre,0.0,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.00,...,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00


#### Let's print each neighborhood along with the top 5 most common venues

In [88]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp =toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.05
2   Restaurant  0.04
3        Hotel  0.04
4          Bar  0.04


----Bathurst Quay----
                 venue  freq
0       Airport Lounge  0.12
1      Airport Service  0.12
2  Rental Car Location  0.06
3      Harbor / Marina  0.06
4     Sculpture Garden  0.06


----Berczy Park----
         venue  freq
0  Coffee Shop  0.09
1     Beer Bar  0.04
2       Bakery  0.04
3   Restaurant  0.04
4  Cheese Shop  0.04


----Brockton----
            venue  freq
0            Café  0.13
1       Nightclub  0.09
2     Coffee Shop  0.09
3  Breakfast Spot  0.09
4             Gym  0.04


----Business reply mail Processing Centre----
                  venue  freq
0           Pizza Place  0.06
1      Recording Studio  0.06
2            Skate Park  0.06
3  Fast Food Restaurant  0.06
4                  Park  0.06


----CN Tower----
                 venue  freq
0       Airport Lounge  0.12
1      Airport Service  0.12
2  Rental Car 

                venue  freq
0         Coffee Shop  0.24
1         Yoga Studio  0.03
2            Beer Bar  0.03
3  Chinese Restaurant  0.03
4  College Auditorium  0.03


----Railway Lands----
                 venue  freq
0       Airport Lounge  0.12
1      Airport Service  0.12
2  Rental Car Location  0.06
3      Harbor / Marina  0.06
4     Sculpture Garden  0.06


----Rathnelly----
                   venue  freq
0            Coffee Shop  0.14
1            Supermarket  0.07
2    Fried Chicken Joint  0.07
3           Liquor Store  0.07
4  Vietnamese Restaurant  0.07


----Regent Park----
         venue  freq
0  Coffee Shop  0.18
1         Park  0.07
2          Pub  0.07
3       Bakery  0.07
4      Theater  0.05


----Richmond----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.05
2   Restaurant  0.04
3        Hotel  0.04
4          Bar  0.04


----Riverdale----
                    venue  freq
0        Greek Restaurant  0.16
1             Coffee Shop  0.09
2      Italian Rest

First, let's write a function to sort the venues in descending order.

In [89]:
#Definefunction to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Restaurant,Hotel,Bar,Gym,Clothing Store,Thai Restaurant,Juice Bar,Concert Hall
1,Bathurst Quay,Airport Lounge,Airport Service,Sculpture Garden,Boutique,Plane,Rental Car Location,Coffee Shop,Boat or Ferry,Harbor / Marina,Bar
2,Berczy Park,Coffee Shop,Bakery,Restaurant,Cheese Shop,Beer Bar,Cocktail Bar,Seafood Restaurant,Farmers Market,Breakfast Spot,Pharmacy
3,Brockton,Café,Breakfast Spot,Nightclub,Coffee Shop,Grocery Store,Burrito Place,Bakery,Intersection,Italian Restaurant,Restaurant
4,Business reply mail Processing Centre,Park,Pizza Place,Skate Park,Brewery,Burrito Place,Farmers Market,Fast Food Restaurant,Restaurant,Butcher,Recording Studio


Run _k_-means to cluster the neighborhood into 5 clusters.

In [93]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 3, 3, 3, 0, 3, 3, 3, 3], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [96]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = neighborhoods
# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

In [97]:
toronto_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Pub,Trail,Health Food Store,Women's Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
73,M4K,East Toronto,The Danforth West,43.679557,-79.352188,3,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Pub,Pizza Place,Lounge
74,M4K,East Toronto,Riverdale,43.679557,-79.352188,3,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Pub,Pizza Place,Lounge
75,M4L,East Toronto,India Bazaar,43.668999,-79.315572,3,Park,Pizza Place,Fast Food Restaurant,Pub,Fish & Chips Shop,Steakhouse,Sushi Restaurant,Burrito Place,Italian Restaurant,Restaurant
76,M4L,East Toronto,The Beaches West,43.668999,-79.315572,3,Park,Pizza Place,Fast Food Restaurant,Pub,Fish & Chips Shop,Steakhouse,Sushi Restaurant,Burrito Place,Italian Restaurant,Restaurant
77,M4M,East Toronto,Studio District,43.659526,-79.340923,3,Coffee Shop,Bakery,Gastropub,Brewery,Café,American Restaurant,Convenience Store,Seafood Restaurant,Cheese Shop,Clothing Store
78,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Coffee Shop,Park,Clothing Store,Yoga Studio,Chinese Restaurant,Bagel Shop,Ice Cream Shop,Bus Line,Spa,Mexican Restaurant
79,M4P,Central Toronto,Davisville North,43.712751,-79.390197,3,Park,Sandwich Place,Dance Studio,Hotel,Food & Drink Shop,Dog Run,Breakfast Spot,Gym / Fitness Center,Department Store,Donut Shop
80,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,3,Coffee Shop,Clothing Store,Yoga Studio,Bagel Shop,Furniture / Home Store,Ice Cream Shop,Fast Food Restaurant,Diner,Mexican Restaurant,Chinese Restaurant
81,M4R,Central Toronto,Lawrence Park,43.715383,-79.405678,3,Coffee Shop,Park,Clothing Store,Yoga Studio,Chinese Restaurant,Bagel Shop,Ice Cream Shop,Bus Line,Spa,Mexican Restaurant


In [98]:
toronto_merged['Cluster Labels']= toronto_merged['Cluster Labels'].fillna(0).astype(int)
#toronto_merged.head() # check the last columns!
toronto_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Pub,Trail,Health Food Store,Women's Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
73,M4K,East Toronto,The Danforth West,43.679557,-79.352188,3,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Pub,Pizza Place,Lounge
74,M4K,East Toronto,Riverdale,43.679557,-79.352188,3,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Pub,Pizza Place,Lounge
75,M4L,East Toronto,India Bazaar,43.668999,-79.315572,3,Park,Pizza Place,Fast Food Restaurant,Pub,Fish & Chips Shop,Steakhouse,Sushi Restaurant,Burrito Place,Italian Restaurant,Restaurant
76,M4L,East Toronto,The Beaches West,43.668999,-79.315572,3,Park,Pizza Place,Fast Food Restaurant,Pub,Fish & Chips Shop,Steakhouse,Sushi Restaurant,Burrito Place,Italian Restaurant,Restaurant
77,M4M,East Toronto,Studio District,43.659526,-79.340923,3,Coffee Shop,Bakery,Gastropub,Brewery,Café,American Restaurant,Convenience Store,Seafood Restaurant,Cheese Shop,Clothing Store
78,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Coffee Shop,Park,Clothing Store,Yoga Studio,Chinese Restaurant,Bagel Shop,Ice Cream Shop,Bus Line,Spa,Mexican Restaurant
79,M4P,Central Toronto,Davisville North,43.712751,-79.390197,3,Park,Sandwich Place,Dance Studio,Hotel,Food & Drink Shop,Dog Run,Breakfast Spot,Gym / Fitness Center,Department Store,Donut Shop
80,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,3,Coffee Shop,Clothing Store,Yoga Studio,Bagel Shop,Furniture / Home Store,Ice Cream Shop,Fast Food Restaurant,Diner,Mexican Restaurant,Chinese Restaurant
81,M4R,Central Toronto,Lawrence Park,43.715383,-79.405678,3,Coffee Shop,Park,Clothing Store,Yoga Studio,Chinese Restaurant,Bagel Shop,Ice Cream Shop,Bus Line,Spa,Mexican Restaurant


## Plot Clusters

#### Use geopy library to get the latitude and longitude values of Tront.

In [22]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
print('Libraries imported.')

Libraries imported.


In [23]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tront are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tront are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [24]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


In [99]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'],toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        ill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine each cluster and determine the discriminating venue categories that distinguish each cluster.

#### Cluster 1

This cluster is located near Airport.

In [100]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0,toronto_merged.columns[[2] + list(range(5,toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
124,CN Tower,0,Airport Lounge,Airport Service,Sculpture Garden,Boutique,Plane,Rental Car Location,Coffee Shop,Boat or Ferry,Harbor / Marina,Bar
125,King and Spadina,0,Airport Lounge,Airport Service,Sculpture Garden,Boutique,Plane,Rental Car Location,Coffee Shop,Boat or Ferry,Harbor / Marina,Bar
126,Railway Lands,0,Airport Lounge,Airport Service,Sculpture Garden,Boutique,Plane,Rental Car Location,Coffee Shop,Boat or Ferry,Harbor / Marina,Bar
127,Harbourfront West,0,Airport Lounge,Airport Service,Sculpture Garden,Boutique,Plane,Rental Car Location,Coffee Shop,Boat or Ferry,Harbor / Marina,Bar
128,Bathurst Quay,0,Airport Lounge,Airport Service,Sculpture Garden,Boutique,Plane,Rental Car Location,Coffee Shop,Boat or Ferry,Harbor / Marina,Bar
129,South Niagara,0,Airport Lounge,Airport Service,Sculpture Garden,Boutique,Plane,Rental Car Location,Coffee Shop,Boat or Ferry,Harbor / Marina,Bar
130,Island airport,0,Airport Lounge,Airport Service,Sculpture Garden,Boutique,Plane,Rental Car Location,Coffee Shop,Boat or Ferry,Harbor / Marina,Bar


#### Cluster 2

This cluster is for visiters to History Museum or Music Venue and has many Sandwich Place, Cafe's.

In [101]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1,toronto_merged.columns[[2] + list(range(5,toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
116,The Annex,1,Sandwich Place,Café,Coffee Shop,History Museum,Liquor Store,Burger Joint,Indian Restaurant,BBQ Joint,Donut Shop,Middle Eastern Restaurant
117,North Midtown,1,Sandwich Place,Café,Coffee Shop,History Museum,Liquor Store,Burger Joint,Indian Restaurant,BBQ Joint,Donut Shop,Middle Eastern Restaurant
118,Yorkville,1,Sandwich Place,Café,Coffee Shop,History Museum,Liquor Store,Burger Joint,Indian Restaurant,BBQ Joint,Donut Shop,Middle Eastern Restaurant
140,Dufferin,1,Pharmacy,Bakery,Grocery Store,Supermarket,Middle Eastern Restaurant,Music Venue,Café,Brewery,Bar,Bank
141,Dovercourt Village,1,Pharmacy,Bakery,Grocery Store,Supermarket,Middle Eastern Restaurant,Music Venue,Café,Brewery,Bar,Bank


#### Cluster 3

This cluster has Parks, Trails, and ethnic food restaurants.

In [102]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2,toronto_merged.columns[[2] + list(range(5,toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
90,Rosedale,2,Park,Trail,Playground,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
114,Forest Hill North & West,2,Park,Sushi Restaurant,Jewelry Store,Trail,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
115,Forest Hill Road Park,2,Park,Sushi Restaurant,Jewelry Store,Trail,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


#### Cluster 4

This cluster has trestaurants and coffee shops.

In [105]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3,toronto_merged.columns[[2] + list(range(5,toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,The Beaches,3,Pub,Trail,Health Food Store,Women's Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
73,The Danforth West,3,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Pub,Pizza Place,Lounge
74,Riverdale,3,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Pub,Pizza Place,Lounge
75,India Bazaar,3,Park,Pizza Place,Fast Food Restaurant,Pub,Fish & Chips Shop,Steakhouse,Sushi Restaurant,Burrito Place,Italian Restaurant,Restaurant
76,The Beaches West,3,Park,Pizza Place,Fast Food Restaurant,Pub,Fish & Chips Shop,Steakhouse,Sushi Restaurant,Burrito Place,Italian Restaurant,Restaurant
77,Studio District,3,Coffee Shop,Bakery,Gastropub,Brewery,Café,American Restaurant,Convenience Store,Seafood Restaurant,Cheese Shop,Clothing Store
78,Lawrence Park,3,Coffee Shop,Park,Clothing Store,Yoga Studio,Chinese Restaurant,Bagel Shop,Ice Cream Shop,Bus Line,Spa,Mexican Restaurant
79,Davisville North,3,Park,Sandwich Place,Dance Studio,Hotel,Food & Drink Shop,Dog Run,Breakfast Spot,Gym / Fitness Center,Department Store,Donut Shop
80,North Toronto West,3,Coffee Shop,Clothing Store,Yoga Studio,Bagel Shop,Furniture / Home Store,Ice Cream Shop,Fast Food Restaurant,Diner,Mexican Restaurant,Chinese Restaurant
81,Lawrence Park,3,Coffee Shop,Park,Clothing Store,Yoga Studio,Chinese Restaurant,Bagel Shop,Ice Cream Shop,Bus Line,Spa,Mexican Restaurant


#### Cluster 5

This cluster has trail and playground.

In [104]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4,toronto_merged.columns[[2] + list(range(5,toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Moore Park,4,Trail,Playground,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
84,Summerhill East,4,Trail,Playground,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
